In [1]:
!aws s3 sync s3://visualneurons.com-logos-outputs /home/ubuntu/data/visualneurons.com-logos-outputs
!aws s3 sync s3://visualneurons.com-logos /home/ubuntu/data/visualneurons.com-logos

download: s3://visualneurons.com-logos-outputs/francesco.pochetti_at_bolt.eu/2023/5/13/13/59/40/output/137cbb3fa86849e89960d44a59063e80.jpeg to ../../data/visualneurons.com-logos-outputs/francesco.pochetti_at_bolt.eu/2023/5/13/13/59/40/output/137cbb3fa86849e89960d44a59063e80.jpeg
download: s3://visualneurons.com-logos-outputs/francesco.pochetti_at_bolt.eu/2023/5/13/13/59/40/output/2aa1963af51e4ad7acb362822b7cec2e.jpeg to ../../data/visualneurons.com-logos-outputs/francesco.pochetti_at_bolt.eu/2023/5/13/13/59/40/output/2aa1963af51e4ad7acb362822b7cec2e.jpeg
download: s3://visualneurons.com-logos-outputs/francesco.pochetti_at_bolt.eu/2023/5/13/13/59/40/output/df89553a2558405589365dc0e9d4bc12.jpeg to ../../data/visualneurons.com-logos-outputs/francesco.pochetti_at_bolt.eu/2023/5/13/13/59/40/output/df89553a2558405589365dc0e9d4bc12.jpeg
download: s3://visualneurons.com-logos-outputs/francesco.pochetti_at_bolt.eu/2023/5/13/13/59/40/output/8ea0a64d5a154f72be22eb427a626299.jpeg to ../../data/vi

In [2]:
from fastcore.all import Path
import os
import pandas as pd

data_path = Path("/home/ubuntu/data")
input_path = data_path / "visualneurons.com-logos"
output_path = data_path / "visualneurons.com-logos-outputs"

filter_users_out = ["pochetti", "lgvaz", "vzz", "lucas"]
filter_dirs_out = ["zipped", "tests", "sagemaker"]

In [3]:
def is_one_of_us(user):
    for f in filter_users_out:
        if f in user:
            return True
    return False

def is_ok_dir(dir_name):
    for f in filter_dirs_out + filter_users_out:
        if f in dir_name:
            return False
    return True

In [4]:
total_users = [user for user in input_path.ls() if not is_one_of_us(str(user))]
user_activity = [user for user in output_path.ls() if is_ok_dir(str(user))]

len(total_users), len(user_activity)

(33, 33)

In [5]:
def list_files_recursive(directory):
    all_files = []
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            all_files.append(Path(file_path))
    return all_files

def extract_time(s):
    s = str(s.path).split("/")[6:-2]
    year = s[0]
    month = s[1]
    day = s[2]
    hour = s[3]
    minute = s[4]
    second = s[5] if len(s) == 6 else "0"
    return int(year), int(month), int(day), int(hour), int(minute), int(second)

def count_files(s):
    files = s.path.parent.ls()
    n_zip_files = len([f for f in files if ".zip" in str(f)])
    n_jpeg_files = len([f for f in files if ".jpeg" in str(f)])
    n_json_files = len([f for f in files if ".json" in str(f)])
    return n_zip_files, n_jpeg_files, n_json_files

files = []
for user in user_activity:
    files += list_files_recursive(user)

df = pd.DataFrame(files, columns=["path"])
df["fname"] = df["path"].apply(lambda x: x.name)
df["email"] = df["path"].apply(lambda x: str(x).split("/")[5].replace("_at_", "@"))
df[["year", "month", "day", "hour", "minute", "second"]] = df.apply(extract_time, axis=1, result_type="expand")
df["extension"] = df["fname"].apply(lambda x: x.split(".")[-1])
df["input_output"] = df["path"].apply(lambda x: "input" if "input" in str(x) else "output")
df[["n_zip_files", "n_jpeg_files", "n_json_files"]] = df.apply(count_files, axis=1, result_type="expand")
df["output_parent"] = df["path"].apply(lambda x: str(x.parent).replace("input", "output"))
df = df[df.month>4]

df.sample(6)

,path,fname,email,year,month,day,hour,minute,second,extension,input_output,n_zip_files,n_jpeg_files,n_json_files,output_parent
471,/home/ubuntu/data/visualneurons.com-logos-outp...,799d71c24d9747f897647403fc1b4ccd.jpeg,sjc@gmail.com,2023,5,2,12,1,0,jpeg,output,1,68,0,/home/ubuntu/data/visualneurons.com-logos-outp...
131,/home/ubuntu/data/visualneurons.com-logos-outp...,d385877c756147b09e970f3d8b96a7cb.jpeg,sparsamu@amazon.com,2023,5,4,16,17,0,jpeg,output,1,62,0,/home/ubuntu/data/visualneurons.com-logos-outp...
1178,/home/ubuntu/data/visualneurons.com-logos-outp...,7e3ca9467bbf466cb3d61caf88f92dea.jpeg,adrienlebret@gmail.com,2023,5,3,7,7,0,jpeg,output,1,60,0,/home/ubuntu/data/visualneurons.com-logos-outp...
876,/home/ubuntu/data/visualneurons.com-logos-outp...,8f9b9588c7114cf49cd2b50204af3d58.jpeg,roger.sindreu@gmail.com,2023,5,2,2,50,0,jpeg,output,1,60,0,/home/ubuntu/data/visualneurons.com-logos-outp...
606,/home/ubuntu/data/visualneurons.com-logos-outp...,7f00ea5a22744e2abd00dc28ec65c816.jpeg,theo@unifai.fr,2023,5,1,22,18,0,jpeg,output,1,74,0,/home/ubuntu/data/visualneurons.com-logos-outp...
721,/home/ubuntu/data/visualneurons.com-logos-outp...,7fde98c3ea0445c283a45640eebc78c6.jpeg,jfigueroa@konfront.mx,2023,5,2,18,36,0,jpeg,output,1,60,0,/home/ubuntu/data/visualneurons.com-logos-outp...


In [6]:
_in = df[df["input_output"] == "input"]
outputs = df[df["input_output"] == "output"].output_parent.unique()
_in["has_output"] = _in.apply(lambda x: True if x.output_parent in outputs else False, axis=1)
assert len(_in.path.unique()) == len(_in)

_out = df[df["input_output"] == "output"]
subs = _in.groupby("email").path.count().to_frame("n_submissions").reset_index()
print(_out.shape)
_out = _out.merge(subs, on="email")
print(_out.shape)
_out["successful_output"] = _out.apply(lambda x: bool(x.n_jpeg_files>=60 and x.n_zip_files), axis=1)

inter = _out.groupby(["output_parent"])[["successful_output"]].min().reset_index()
_in = _in.merge(inter, on="output_parent", how="left")
_in.successful_output.fillna(False, inplace=True)

(1349, 15)
(1349, 16)


/tmp/ipykernel_22242/1346639450.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _in["has_output"] = _in.apply(lambda x: True if x.output_parent in outputs else False, axis=1)


In [7]:
print(f"{len(_in)} unique submissions from {len(_in.email.unique())} users, of which {_in.successful_output.sum()} are successful")

44 unique submissions from 32 users, of which 21 are successful


In [8]:
s = _in.groupby("email").agg({"path": "count", "successful_output": "sum"}).sort_values("path", ascending=False)
s.columns = ["n_submissions", "n_successful_submissions"]
s["success_rate"] = s["n_successful_submissions"] / s["n_submissions"]
s

,n_submissions,n_successful_submissions,success_rate
email,,,
sparsamu@amazon.com,8,4,0.5
adrienlebret@gmail.com,4,4,1.0
mmcdonald@cars.com,2,0,0.0
khaliladib11@gmail.com,2,0,0.0
a.shiban@gmail.com,1,0,0.0
tony.walasik@gmail.com,1,1,1.0
tom.godden@gmail.com,1,0,0.0
thesarthakrastogi@gmail.com,1,0,0.0
theo@unifai.fr,1,1,1.0


In [9]:
_in.loc[_in.successful_output==False, "path"].to_list()

[Path('/home/ubuntu/data/visualneurons.com-logos-outputs/tom.godden_at_gmail.com/2023/5/2/16/27/input/foo.json'),
 Path('/home/ubuntu/data/visualneurons.com-logos-outputs/mmcdonald_at_cars.com/2023/5/3/14/8/input/Carslogo.json'),
 Path('/home/ubuntu/data/visualneurons.com-logos-outputs/mmcdonald_at_cars.com/2023/5/3/20/57/input/Carslogo.json'),
 Path('/home/ubuntu/data/visualneurons.com-logos-outputs/cheema8raman_at_gmail.com/2023/5/3/20/19/input/Screenshot_2023_05_03_at_22.json'),
 Path('/home/ubuntu/data/visualneurons.com-logos-outputs/sparsamu_at_amazon.com/2023/5/5/16/42/input/Picture3.json'),
 Path('/home/ubuntu/data/visualneurons.com-logos-outputs/sparsamu_at_amazon.com/2023/5/5/22/16/input/Picture3.json'),
 Path('/home/ubuntu/data/visualneurons.com-logos-outputs/sparsamu_at_amazon.com/2023/5/4/22/34/input/Picture3.json'),
 Path('/home/ubuntu/data/visualneurons.com-logos-outputs/sparsamu_at_amazon.com/2023/5/3/20/22/input/Picture2.json'),
 Path('/home/ubuntu/data/visualneurons.co